In [1]:
import pandas as pd
import numpy as np
import json
import csv
import requests
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy import MetaData

In [2]:
def connect_db(database='fees'):
    user='root'
    password='240699'
    host='localhost'
    database=database
    

    # Connect to the database
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
    try:
        connection = engine.connect()
        return connection
    except Exception as e:
        raise e

In [3]:
response = requests.get('https://api.llama.fi/overview/fees?excludeTotalDataChart=false&excludeTotalDataChartBreakdown=false&dataType=dailyFees')
data = response.json()

In [11]:
data_chart = data['totalDataChart']
breakdown = data['totalDataChartBreakdown']


In [34]:
fees_historic = pd.DataFrame(data_chart, columns=['date', 'fees'])
fees_historic['date'] = pd.to_datetime(fees_historic['date'], unit='s')

In [35]:
# Calculate percentage changes for different time intervals
fees_historic['1d_change'] = fees_historic['fees'].pct_change() * 100
fees_historic['7d_change'] = fees_historic['fees'].pct_change(periods=7) * 100
fees_historic['30d_change'] = fees_historic['fees'].pct_change(periods=30) * 100
fees_historic['90d_change'] = fees_historic['fees'].pct_change(periods=90) * 100
fees_historic['180d_change'] = fees_historic['fees'].pct_change(periods=180) * 100
fees_historic['1y_change'] = fees_historic['fees'].pct_change(periods=365) * 100

#### Rolling Statistical

In [36]:
fees_historic['rolling_mean_10'] = fees_historic['fees'].rolling(window=10).mean()
fees_historic['rolling_mean_25'] = fees_historic['fees'].rolling(window=25).mean()
fees_historic['rolling_mean_50'] = fees_historic['fees'].rolling(window=50).mean()
fees_historic['rolling_mean_100'] = fees_historic['fees'].rolling(window=100).mean()


In [37]:
fees_historic['rolling_std_10'] = fees_historic['fees'].rolling(window=10).std()
fees_historic['rolling_std_25'] = fees_historic['fees'].rolling(window=25).std()
fees_historic['rolling_std_50'] = fees_historic['fees'].rolling(window=50).std()
fees_historic['rolling_std_100'] = fees_historic['fees'].rolling(window=100).std()


In [38]:
fees_historic['rolling_min_10'] = fees_historic['fees'].rolling(window=10).min()
fees_historic['rolling_min_25'] = fees_historic['fees'].rolling(window=25).min()
fees_historic['rolling_min_50'] = fees_historic['fees'].rolling(window=50).min()
fees_historic['rolling_min_100'] = fees_historic['fees'].rolling(window=100).min()

In [39]:
fees_historic['rolling_max_10'] = fees_historic['fees'].rolling(window=10).max()
fees_historic['rolling_max_25'] = fees_historic['fees'].rolling(window=25).max()
fees_historic['rolling_max_50'] = fees_historic['fees'].rolling(window=50).max()
fees_historic['rolling_max_100'] = fees_historic['fees'].rolling(window=100).max()

#### Volatility

In [40]:
fees_historic['historical_volatility'] = fees_historic['fees'].pct_change().rolling(window=10).std() * np.sqrt(252)


In [41]:
fees_historic['ema_12'] = fees_historic['fees'].ewm(span=12, adjust=False).mean()
fees_historic['ema_26'] = fees_historic['fees'].ewm(span=26, adjust=False).mean()
fees_historic['macd'] = fees_historic['ema_12'] - fees_historic['ema_26']
fees_historic['signal_line'] = fees_historic['macd'].ewm(span=9, adjust=False).mean()

In [42]:
delta = fees_historic['fees'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
fees_historic['rsi'] = 100 - (100 / (1 + rs))

In [43]:
fees_historic['rate_of_change'] = fees_historic['fees'].pct_change() * 100
fees_historic['momentum'] = fees_historic['fees'].diff()

In [44]:
fees_historic['sma_20'] = fees_historic['fees'].rolling(window=20).mean()
fees_historic['upper_band'] = fees_historic['sma_20'] + 2 * fees_historic['fees'].rolling(window=20).std()
fees_historic['lower_band'] = fees_historic['sma_20'] - 2 * fees_historic['fees'].rolling(window=20).std()


In [45]:
fees_historic['stoch_oscillator'] = 100 * ((fees_historic['fees'] - fees_historic['fees'].rolling(window=14).min()) / 
                                           (fees_historic['fees'].rolling(window=14).max() - fees_historic['fees'].rolling(window=14).min()))


In [47]:
with connect_db() as connection:
    fees_historic.to_sql( 'historic', con = connection, if_exists = 'replace' )
    

In [49]:
df = pd.DataFrame(breakdown, columns=['date', 'id'])
df['date'] = pd.to_datetime(df['date'], unit='s')
df = df.set_index('date')

In [51]:
df = pd.DataFrame(df['id'].tolist(), index=df.index)

dfs 

In [69]:
dfs = []
count = 0
with connect_db('fees') as connection:
    for column in df.columns:
        # Create an individual DataFrame for each column
        individual_df = pd.DataFrame({
            'date': df.index,
            'id': column,
            'fees': df[column],
            '1d_change': df[column].pct_change() * 100,
            '7d_change': df[column].pct_change(periods=7) * 100,
            '30d_change': df[column].pct_change(periods=30) * 100,
            '90d_change': df[column].pct_change(periods=90) * 100,
            '180d_change': df[column].pct_change(periods=180) * 100,
            '1y_change': df[column].pct_change(periods=365) * 100,

            'ema_10': df[column].ewm(span=10, adjust=False).mean(),
            'ema_25': df[column].ewm(span=25, adjust=False).mean(),
            'ema_50': df[column].ewm(span=50, adjust=False).mean(),
            'ema_100': df[column].ewm(span=100, adjust=False).mean(),

            'rolling_std_10': df[column].rolling(window=10).std(),
            'rolling_std_25': df[column].rolling(window=25).std(),
            'rolling_std_50': df[column].rolling(window=50).std(),
            'rolling_std_100': df[column].rolling(window=100).std(),
            
            'rolling_min_10': df[column].rolling(window=10).min(),
            'rolling_max_10': df[column].rolling(window=10).max(),
            'rolling_min_25': df[column].rolling(window=25).min(),
            'rolling_max_25': df[column].rolling(window=25).max(),
            'rolling_min_50': df[column].rolling(window=50).min(),
            'rolling_max_50': df[column].rolling(window=50).max(),
            'rolling_min_100': df[column].rolling(window=100).min(),
            'rolling_max_100': df[column].rolling(window=100).max(),

            'historical_volatility': df[column].pct_change().rolling(window=10).std() * np.sqrt(252),
            
            'ema_12': df[column].ewm(span=12, adjust=False).mean(),
            'ema_26': df[column].ewm(span=26, adjust=False).mean(),
            'macd': df[column].ewm(span=12, adjust=False).mean() - df[column].ewm(span=26, adjust=False).mean(),
            'signal_line': (df[column].ewm(span=12, adjust=False).mean() - df[column].ewm(span=26, adjust=False).mean()).ewm(span=9, adjust=False).mean(),

            'rsi': 100 - (100 / (1 + df[column].diff().where(df[column].diff() > 0, 0).rolling(window=14).mean() / df[column].diff().where(df[column].diff() < 0, 0).rolling(window=14).mean())),

            'momentum': df[column].diff(),
            
            'sma_20': df[column].rolling(window=20).mean(),
            'upper_band': df[column].rolling(window=20).mean() + 2 * df[column].rolling(window=20).std(),
            'lower_band': df[column].rolling(window=20).mean() - 2 * df[column].rolling(window=20).std(),


            'log_return': np.log(df[column] / df[column].shift(1)) * 100,
        })
        name = individual_df.id
        count+=1
        individual_df.to_sql( name, con = connection, if_exists = 'replace' )
        print(name + 'inserted.' + len(df.columns-count))
        # Append the individual DataFrame to the list
        dfs.append(individual_df)


KeyError: 'sma_20'

In [57]:
dfs[1]

,date,id,fees,ema_10,ema_25,daily_return,log_return
date,,,,,,,
2019-10-11,2019-10-11,Compound,14413.043083,14413.043083,14413.043083,NaN,NaN
2019-10-12,2019-10-12,Compound,14183.369765,14371.284298,14395.375905,-1.593510,-1.606343
2019-10-13,2019-10-13,Compound,14280.334048,14354.747889,14386.526531,0.683648,0.681321
2019-10-14,2019-10-14,Compound,14191.084244,14324.990863,14371.492509,-0.624984,-0.626945
2019-10-15,2019-10-15,Compound,14391.165232,14337.022566,14373.005796,1.409906,1.400060
...,...,...,...,...,...,...,...
2023-12-29,2023-12-29,Compound,116095.060220,122681.894363,119099.602224,3.936267,3.860771
2023-12-30,2023-12-30,Compound,100529.539997,118654.193569,117671.135899,-13.407565,-14.395773
2023-12-31,2023-12-31,Compound,93262.206814,114037.468704,115793.525969,-7.229052,-7.503666


In [60]:
data.keys()

dict_keys(['totalDataChart', 'totalDataChartBreakdown', 'protocols', 'allChains', 'chain', 'total24h', 'total48hto24h', 'total7d', 'total14dto7d', 'total60dto30d', 'total30d', 'total1y', 'average1y', 'change_1d', 'change_7d', 'change_1m', 'totalVolume7d', 'totalVolume30d', 'change_7dover7d', 'change_30dover30d', 'breakdown24h', 'dailyRevenue', 'dailyUserFees', 'dailyHoldersRevenue', 'dailySupplySideRevenue', 'dailyProtocolRevenue', 'dailyBribesRevenue', 'dailyTokenTaxes'])

In [67]:
for i in data['protocols']:
    print (i)

{'defillamaId': '2685', 'name': 'Arbitrum Exchange V2', 'disabled': False, 'displayName': 'Arbitrum Exchange V2', 'module': 'ArbitrumExchange', 'category': 'Dexes', 'logo': 'https://icons.llamao.fi/icons/protocols/arbitrum-exchange-v2.jpg', 'change_1d': 40, 'change_7d': 139, 'change_1m': 146.18, 'change_7dover7d': 21.03, 'change_30dover30d': 11.14, 'total24h': 7.869158702924401, 'total48hto24h': 5.620886304903213, 'total7d': 54.1244658821605, 'total30d': 207.4377805243843, 'total14dto7d': 44.721440641999074, 'total60dto30d': 186.65385170565978, 'total1y': 0, 'average1y': 0, 'totalAllTime': 238541.339773482, 'breakdown24h': {'arbitrum': {'v2': 7.869158702924401}}, 'chains': ['Arbitrum'], 'protocolType': 'protocol', 'methodologyURL': 'https://github.com/DefiLlama/dimension-adapters/blob/master/dexs/ArbitrumExchange', 'methodology': {'UserFees': 'User pays 0.25% fees on each swap.', 'Fees': 'Swap fees paid by users', 'Revenue': 'Percentage of swap fees going to treasury and/or token holde